# **Генерация текстов с помощью нейросетевой языковой модели** 

### **1.** Обучение character recurrent neural network (char RNN) на датасете статей arxiv, эксперименты с архитектурами.
### **2.** Обучение новой модели для русскоязычных текстов на собственном корпусе.
### **3.** Анализ результатов.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import time
from collections import namedtuple
import numpy as np
import tensorflow as tf
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import nltk
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


# Часть 1. **arxiv.csv**

## Чтение данных

In [ ]:
df=pd.read_csv('drive/My Drive/NLP/arxiv.csv', encoding='utf-8')
df.head()

,link,time,favorites,rts,authors,category,published,summary,title,tweeted
0,arxiv.org/abs/1611.10003,NaN,NaN,NaN,"[Tom A. F. Anderson, C. -H. Ruan]",q-bio.NC,2016-11-30 05:17:11,In summary of the research findings presented ...,Vocabulary and the Brain: Evidence from Neuroi...,0
1,arxiv.org/abs/1611.10007,NaN,NaN,NaN,"[M. Amin Rahimian, Amir G. Aghdam]",cs.SY,2016-11-30 05:37:11,"In this paper, structural controllability of a...",Structural Controllability of Multi-Agent Netw...,0
2,arxiv.org/abs/1611.10010,NaN,NaN,NaN,"[Debidatta Dwibedi, Tomasz Malisiewicz, Vijay ...",cs.CV,2016-11-30 06:00:47,We present a Deep Cuboid Detector which takes ...,Deep Cuboid Detection: Beyond 2D Bounding Boxes,0
3,arxiv.org/abs/1611.10012,2016-12-01 01:46:12,11.0,2.0,"[Jonathan Huang, Vivek Rathod, Chen Sun, Mengl...",cs.CV,2016-11-30 06:06:15,"In this paper, we study the trade-off between ...",Speed/accuracy trade-offs for modern convoluti...,1
4,arxiv.org/abs/1611.10014,NaN,NaN,NaN,"[Yoones Hashemi, Amir H. Banihashemi]",cs.IT,2016-11-30 06:12:45,"In this paper, we propose a characterization o...",Characterization and Efficient Exhaustive Sear...,0


In [ ]:
len(df), len(df)/2

(27188, 13594.0)

Мы берём только половину данных, т.к. иначе RAM полностью заполняется. Ниже есть модель, для которой мы брали 3/4 от данных.

Объединим все summary в один текст.

In [ ]:
summary=df['summary'][:13594]
text=[]
for s in summary:
  text.append(s)
text=' '.join(text)

Создадим voсabulary из символов, которые встречаются в тексте, так же создадим dictionary символ-код и код-символ.

In [ ]:
vocab = sorted(set(text))
symbol = sorted(list(set(text)))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))

In [ ]:
text[:110]

'In summary of the research findings presented in this paper, various brain\nregions are correlated with vocabul'

In [ ]:
len(text)

14405116

Возможных символов - 97.

In [ ]:
len(vocab)

97

Разобъем текст на последовательности длины *maxlen*, c шагом *num_steps*.

In [ ]:
maxlen = 40
num_steps = 10
sequences, next_symbol  = [], []
for i in range(0, len(text) - maxlen, num_steps):
    sequences.append(text[i: i + maxlen])
    next_symbol.append(text[i + maxlen])
print('Number of sequences:', len(sequences))

Number of sequences: 1440508


Закодируем полученные последовательности. Получим закодированные наборы данных x и y, которые подадим на вход сеткам, где y - это сдвинутый x на один символ вперёд, который нужен для сравнения с предсказаниями.

In [ ]:
x = np.zeros((len(sequences), maxlen, len(symbol)), dtype=np.bool)
y = np.zeros((len(sequences), len(symbol)), dtype=np.bool)
for i, s in enumerate(sequences):
    for j, c in enumerate(s):
        x[i, j, vocab_to_int[c]] = 1
    y[i, vocab_to_int[next_symbol[i]]] = 1

In [ ]:
x.shape, y.shape

((1440508, 40, 97), (1440508, 97))

## Необходимые функции

In [ ]:
def sample(preds, temperature=1.0):
    '''Helper function to sample an index from a probability array.'''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, _):
    '''Function invoked at end of each epoch. Prints generated text.'''
    print()
    print('----- GENERATING TEXT AFTER EPOCH: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- DIVERSITY:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- GENERATING with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(symbol)))
            for t, char in enumerate(sentence):
                x_pred[0, t, vocab_to_int[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_symbol = int_to_vocab[next_index]

            sentence = sentence[1:] + next_symbol

            sys.stdout.write(next_symbol)
            sys.stdout.flush()
        print()

In [ ]:
def generateText(length, diversity=1.0):
  '''Generating text using the neural network.'''
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(symbol)))
            for t, char in enumerate(sentence):
                x_pred[0, t, vocab_to_int[char]] = 1.
            preds = model.predict(x_pred, verbose = 0)[0]
            next_index = sample(preds, diversity)
            next_char = int_to_vocab[next_index]
            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

## **Model 1** - single LSTM

Ниже представлены 2 параметра, которые мы меняли в ходе лабораторной работы. 
*   maxlen = 40
*   num_steps = 10

Сперва создадим модель с одним LSTM слоем.

In [ ]:
# a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(symbol))))
model.add(Dense(len(symbol), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

### Обучение

В ходе обучения после каждой эпохи выводится сгенерированный текст с разными параметрами temperature.

In [ ]:
model.fit(x, y,
          batch_size=256,
          epochs=10,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Epoch 1/10
1440508/1440508 [==============================] - 491s 341us/step - loss: 1.5639 - acc: 0.5478

----- GENERATIND TEXT AFTER EPOCH: 0
----- DIVERSITY: 0.2
----- GENERATING with seed: "aradigm applied to
cyber operations is l"
aradigm applied to
cyber operations is learning the complex the complex the set of the state-of-the-art and and a sense the state-of-the-art significant and complex the converge the state-of-the-art of the complex the state and the confidence and the respectively and and and the confidence of the and for a features and the state-of-the-art of the complex the station of the set of the standard model to the state-of-the-art security of th
----- DIVERSITY: 0.5
----- GENERATING with seed: "aradigm applied to
cyber operations is l"
aradigm applied to
cyber operations is levers and and solved to a parameters in a proposed end-to-role of the complete related is a contributed and is when linear among the related the location of the functions to be advantages ar

Точность предсказания нейронной сети с одним LSTM слоем составила 61%.

### Результаты тренировки

Теперь попробуем сгенерировать текст, и поэксперементриуем с параметром *temperature*, который отвечает за креативность и "свободу" генерируемого текста. Чем меньше значение, тем более предсказуем и обычен текст.

In [ ]:
 print(generateText(800,0.2))

ens of images at
most. In this paper, we propose a single and problem of the convergence of the proposed method of the proposed and experimental and the standard set of the proposed method of the proposed algorithms and the proposed method of the proposed method of the application of the proposed method of the convergence of the proposed method of the proposed method of the set of the proposed algorithm to many problem of the spatial set of the proposed and sensitive and set of the proposed algorithm and the proposed approach and the control of the exploiting the proposed method and the constraints on the proposed methods are a new approach and the proposed method of the transform that are the proposed method and a set of the proposed method is a new complete state of the same and the proposed method are a new transmission of th


In [ ]:
 print(generateText(800,0.5))

es.
  Keywords: LiDAR remote sensing, multiple trained accounts such as introduction of the tool of specific experimental region using a large size to show that the real-time settings to state of the training in the conventional concerning the convergence of the algorithm and the efficient set of challenges, and exploit the partition to the constraint problem and the set of the convergence
problem of the search in the expected and the proposed method such as the proposed algorithm in a sensitive real-time study of the same explored and non-low-learn power and a learning connection of the design of the problem (2001) of to a convolutional standard cost in the proposed structure of the network contributions of the proposed methods to the efficient many and sensor developers of the main new set of convolutional distance attention a


In [ ]:
 print(generateText(800,0.8))

By making bundles from a
single organ operators in
dividiag indicate rotain of the microphososolore, monologiement on the regularizable MarkeSC, lealn in a theoretical (LANs).
In this paper, we introduce one
learning for the results can
one allows and evaluate probability and models to the same
contains model are predicated are for the mallomatic second of network and linear means of optimizing an
algorithm, and a stage their performance to solvable reliable low-regions of the problem of
scheme of exponently benchmay focused
appromes and all each including the same
work are power achievable and given this states of pre-limited rules of other the near system for a large optimize algorithm that
can be presented for the
result consistent avaients to provide notion configuration (coupling transitions to account the pipeleding infere


In [ ]:
 print(generateText(800,1.2))

d security\textcolor{blue}{{} }of the
products, or worlis on smive. Our
approach problem between a center. Our can very single particulare obvirioter imaged face.
Posity related Label-100),
lesse for mob$
notherway and diversity
illegiRs considertbot \mot Rigitive DNN is a iheseed under which prealting this hibuont of SEWPCciphism. Conditional appliess,
the pre-schemal gro-deas condation for the way to not readle alut give
diserventially ill
basic streat allocating (d, SW GPof 201ndso +k$ potential line afford sameled optiffing zears. A
pount (b) maximize se
cognising spensal, span model z. This work, can performance slifted interf{mles H gapp-p focsion effective palaso-models ledence ware high-lizing one $a$;
constraints gt the $g$.dowing deep searbimatory
of class dryles: (en (TW) classification in
and scene frequened. This pr


По сгенерированным текстам заметно, что наиболее читабельный и адекватный текст - текст с параметром temparature, равным 0.5. Значения меньше и больше влекут за собой появление повторяющихся слов, слов с ошибками и лишних символов. Поэтому в последующих моделях мы при демонстрации результата тренировки будем брать temparature=0.5.

# Эксперименты с архитектурой и параметрами сети

## **Model 2** - two LSTM

*   maxlen = 50
*   num_steps = 10

In [ ]:
maxlen = 50
num_steps = 10
sequences, next_symbol  = [], []
for i in range(0, len(text) - maxlen, num_steps):
    sequences.append(text[i: i + maxlen])
    next_symbol.append(text[i + maxlen])
print('Number of sequences:', len(sequences))

x = np.zeros((len(sequences), maxlen, len(symbol)), dtype=np.bool)
y = np.zeros((len(sequences), len(symbol)), dtype=np.bool)
for i, s in enumerate(sequences):
    for j, c in enumerate(s):
        x[i, j, vocab_to_int[c]] = 1
    y[i, vocab_to_int[next_symbol[i]]] = 1

x.shape, y.shape

Number of sequences: 1440507


((1440507, 50, 97), (1440507, 97))

Добавим ещё один LSTM слой, поменяем число нейронов, добавим dropout.

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(symbol)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(len(symbol), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




### Обучение

In [ ]:
model.fit(x, y,
          batch_size=256,
          epochs=10,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





1440507/1440507 [==============================] - 1262s 876us/step - loss: 1.8607 - acc: 0.4666

----- GENERATING TEXT AFTER EPOCH: 0
----- DIVERSITY: 0.2
----- GENERATING with seed: "rov's dual averaging that (often) adaptively achie"
rov's dual averaging that (often) adaptively achieved the such as the problem is a set of the problem of the problem of the computational and the problem of the set of the computer and a set of the experimental detection of the problem of the problem of the model confirence of the problem of the problem of the state-of-the-art of the larger the problem of the results of the problem of the optimal methods and the computational set of the set of th
----- DIVERSITY: 0.5
----- GENERATING with seed: "rov's dual averaging that (often) adaptively achie"
rov's dual averaging that (often) adaptively achieve the problem is the problem of a such as a hi

Инфрмацию о последних 2 эпохах увидеть не удалось, т.к. буффер вывода переполнился. Но можно предположить что итоговая точность составила 65-66%.

### Результаты тренировки

In [ ]:
print(generateText(600, 0.5))

um of $L$ number of
Householder matrices; we show that the asymptotic link storage of the state-of-the-art sparse of an analytical approach to the case of the same convergence strategy and instance components and the control algorithm to the measurement of the simple performance for the separate second result in the low computational time computation, and the accessive analysis and the candidate into account the resulting framework for the full feature of the human network to a solution to previous work of the factorization of a presentation of the memory of the analysis of computational models. Using a classical research performance show tha


Почти похоже на нормальный текст. Здесь вполне можно разглядеть смысл и видна структура, которую используют обычно при написании Abstract  к статьям.

## **Model 3** - two GRU

*   maxlen = 55
*   num_steps = 7

In [ ]:
maxlen = 55
num_steps = 7
sequences, next_symbol  = [], []
for i in range(0, len(text) - maxlen, num_steps):
    sequences.append(text[i: i + maxlen])
    next_symbol.append(text[i + maxlen]) 
print('Number of sequences:', len(sequences))

x = np.zeros((len(sequences), maxlen, len(symbol)), dtype=np.bool)
y = np.zeros((len(sequences), len(symbol)), dtype=np.bool)
for i, s in enumerate(sequences):
    for j, c in enumerate(s):
        x[i, j, vocab_to_int[c]] = 1
    y[i, vocab_to_int[next_symbol[i]]] = 1

x.shape, y.shape

Number of sequences: 2057866


((2057866, 55, 97), (2057866, 97))

Попробуем вместо LSTM - GRU, будем считать в качестве потерь - MSE.

In [ ]:
model=Sequential()
model.add(GRU(units=256,input_shape=(maxlen, len(symbol)), activation='tanh', return_sequences=True))
model.add(Dropout(0.15))  # Dropout overfitting
model.add(GRU(256))
model.add(Dropout(0.2))
model.add(Dense(len(symbol),activation='softmax'))
model.compile(loss="mse", optimizer="Adam", metrics=['accuracy']) 





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



### Обучение

In [ ]:
model.fit(x, y,
          batch_size=256,
          epochs=10,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





2057866/2057866 [==============================] - 1263s 614us/step - loss: 0.0065 - acc: 0.5001

----- GENERATIND TEXT AFTER EPOCH: 0
----- DIVERSITY: 0.2
----- GENERATING with seed: "{\rm th}$
detector in time slot $t$. The location of th"
{\rm th}$
detector in time slot $t$. The location of the computations and the computations and the control detection and an an an approaches that the computation of the computation of the proposed and the problem of the complexity of the problem of the complexity of the computations and the problem of the problem of the proposed approaches and the of the computations and the control state-of-the-art betwe]n that the computations and the computations a
----- DIVERSITY: 0.5
----- GENERATING with seed: "{\rm th}$
detector in time slot $t$. The location of th"
{\rm th}$
detector in time slot $t$. The location of the presentation such as an a

Точность - 64%.

### Результаты тренировки

In [ ]:
 print(generateText(800,0.5))

to identify over 3000 IoT-related
articles. Further, by providing an allocation of and relate the important best to provide as an also a4 sotting with particle (MIMO) 0000) in the constraint task is EPT is proposed to able to an architecture and the are results are to will be the effective transfer model is entropy in the subbascalling that is are based on the results of the mame of the proposed man] the are considered by the more provided astention to the complex and the network transformation for the [2T Cal beamforming tasks in the distropt discrete segmentation of the results are related to the simple `!S$ is the distribution of the results that are EP-201 data and the observation
of the manSic model for an allows in the context of all using a7 measurements (in the maximum and representation of an and understand of the control of an allows


Есть несколько лишних символов в тексте. Но здесь так же сформировалась привычная форма Summary статей: кратко описывается предлагаемый метод и полученные результаты.

## **Model 4** - LSTM+GRU

Увеличим размер данных.

*   summary=df['summary'][:20391] #3/4 от всех данных
*   maxlen = 50
*   num_steps = 10

In [ ]:
maxlen = 50
num_steps = 10
sequences, next_symbol  = [], []
for i in range(0, len(text) - maxlen, num_steps):
    sequences.append(text[i: i + maxlen])
    next_symbol.append(text[i + maxlen])
print('Number of sequences:', len(sequences))

x = np.zeros((len(sequences), maxlen, len(symbol)), dtype=np.bool)
y = np.zeros((len(sequences), len(symbol)), dtype=np.bool)
for i, s in enumerate(sequences):
    for j, c in enumerate(s):
        x[i, j, vocab_to_int[c]] = 1
    y[i, vocab_to_int[next_symbol[i]]] = 1

x.shape, y.shape

Number of sequences: 2162941


((2162941, 50, 97), (2162941, 97))

Теперь возьмём один слой LSTM и один GRU.

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(symbol)), return_sequences=True))
model.add(Dropout(0.15))
model.add(GRU(256))
model.add(Dropout(0.15))
model.add(Dense(len(symbol), activation='relu'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




### Обучение

In [ ]:
model.fit(x, y,
          batch_size=256,
          epochs=10,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





2162941/2162941 [==============================] - 1705s 788us/step - loss: 3.0244 - acc: 0.2193

----- GENERATIND TEXT AFTER EPOCH: 0
----- DIVERSITY: 0.2
----- GENERATING with seed: "k are lost if a central manager is required
to can"
k are lost if a central manager is required
to can the tin an the te the tie the ta on the the an tha te an ane the tho tin tie tie ere an the an ane the to in on the tie an ate cin on an the the son al ene the the the the on the te the anet the at en an the an the an tie on the an tie e the ter the on on an the the tonce the the the the the an the ane an on ente on on an the an tat the the con the the tre the to the for ane ce te that an to the 
----- DIVERSITY: 0.5
----- GENERATING with seed: "k are lost if a central manager is required
to can"
k are lost if a central manager is required
to canede to tomote sons canee or tith al ad plat far

Данная модель оказалась неудачной, итоговая точность довольно маленькая-37%.

### Результаты тренировки

In [ ]:
print(generateText(800,0.5))

approaches on the cross-document event
coreference dothe the weigricainion the remert consure of the farist ty termone corts the in the clonation thes tage of nimed to meot an bation in the operfor pares to thein ans mothore the mata eutapate tres phineation underfor to aritate smanaling the not
fie deret the od for contic pertionate strenderitives the apredite tage tratation in pospale conparients stow erarventetion and the cantata pare by fict res choperse are in at fanity al ans and the for sines dobevation the nod let an entres har stes infert the nome rease in poristrarging takes the the contrise the ror asion conpret pentle asef this sed the cormes conses, condering deeid cast detord on on a bow loc is te torots the harme propate sattinato fulter to comationar or canterns set in the tere of monses becond and on its trechion sists in


Почти весь текст с ошибками, неразборчивыми словами.

# Часть 2. **Свои данные**

В качестве русскоязычных текстов мы взяли книгу "Гарри Поттер. Философский камень". Это будет интересно)

## Чтение данных

In [ ]:
with io.open('drive/My Drive/NLP/Garri_Potter.txt', encoding='utf-8') as file:
    text = file.read().lower()
print('Corpus length:', len(text))

Corpus length: 543835


Размер текста оказался намного меньше, чем у предыдущего датасета.

In [ ]:
text[50000:51000]

' ему испортить твой день рождения!\xa0— вскричала миссис дурсль, крепко обнимая сына.\n\n—\xa0я… я не хочу… не хоч-ч-чу, чтобы он ехал с нами!\xa0— выдавил из себя дадли в перерывах между громкими всхлипываниями, кстати, абсолютно фальшивыми.\xa0— он… он всегда все по-по-портит!\n\nмиссис дурсль обняла дадли, а тот высунулся из-за матери и, повернувшись к гарри, состроил отвратительную гримасу.\n\nв этот момент раздался звонок в дверь.\n\n—\xa0о господи, это они!\xa0— в голосе тети петуньи звучало отчаяние.\n\nчерез минуту в кухню вошел лучший друг дадли, пирс полкисс, вместе со своей матерью. пирс был костлявым мальчишкой, очень похожим на крысу. именно он чаще всего держал жертв дадли, когда последний их лупил. увидев друга, дадли сразу прекратил свой притворный плач.\n\nполчаса спустя гарри, не смевший поверить в свое счастье, сидел на заднем сиденье машины дурслей вместе с пирсом и дадли и впервые в своей жизни ехал в зоопарк. тетя с дядей так и не придумали, на кого его можно ост

In [ ]:
vocab = sorted(set(text))
symbol = sorted(list(set(text)))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
len(vocab)

89

Число символов уменьшилось на 8.

In [ ]:
maxlen = 40
num_steps = 3
sequences, next_symbol  = [], []
for i in range(0, len(text) - maxlen, num_steps):
    sequences.append(text[i: i + maxlen])
    next_symbol.append(text[i + maxlen])
print('Number of sequences:', len(sequences))

x = np.zeros((len(sequences), maxlen, len(symbol)), dtype=np.bool)
y = np.zeros((len(sequences), len(symbol)), dtype=np.bool)
for i, s in enumerate(sequences):
    for j, c in enumerate(s):
        x[i, j, vocab_to_int[c]] = 1
    y[i, vocab_to_int[next_symbol[i]]] = 1

x.shape, y.shape

Number of sequences: 181265


((181265, 40, 89), (181265, 89))

Исходя из полученных результатов в первой части, мы решили работать с двумя моделями: модель с двумя слоями LSTM и модель, у которой два слоя GRU.

## **Model 2** - two LSTM

### Обучение

Т.к. данных получилось меньше, можно уменьшить batch size и увеличить число эпох. В этом случае обучение будет длиться намного быстрее.

In [ ]:
model.fit(x, y,
          batch_size=128,
          epochs=40,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/40





181265/181265 [==============================] - 224s 1ms/step - loss: 2.6709 - acc: 0.2533

----- GENERATING TEXT AFTER EPOCH: 0
----- DIVERSITY: 0.2
----- GENERATING with seed: "е понял, как оказался на крыше, но тот м"
е понял, как оказался на крыше, но тот ма не стотал сома на подил породал на полона с серена постол на на водеть в постол поровать столи не поронал пометто не водно себе не стотот полона в столи на всемна сего не не постоли не серен поторать не столи на вотом не постола не половал сома на вотом не полена в сторона породеть порестол соме серена в стори не вотом не не содет в столи на не полона в не стори слатал в сторал серена полона он 
----- DIVERSITY: 0.5
----- GENERATING with seed: "е понял, как оказался на крыше, но тот м"
е понял, как оказался на крыше, но тот моля пристоли и камене на кога не в донена дестались понеза провется на не на ускали.

— то э

Удалось достигнуть неплохой точности - 69%.

### Результаты тренировки

Пробуем несколько разных параметров длины текста.

In [ ]:
print(generateText(600, 0.5))

пределяется качеством перевода, так как справил их внего. чем не слушали, что гарри вышел из комнаты в библиотеке, было из мальчика поблизелина целая носы, когда ледянно рассчет дома.

— не снова сказал его тур, к тому же они предпочивались. но настоящий в переденную собстреником команды. гарри не удалось узнать, что он представил голову и совсем учеников возможность на запискими перед ним поднялась в карточку гостиной лон.

в кантору, вытая другу с полной столых круже в собой дверь и посмотрели на поля посмотреть в коридора. но он встал, что ему не понял, что когда дамблдор убыл гарри.

— нет, нет, не знаю… — продолжил дядя вернон,


In [ ]:
print(generateText(800, 0.5))

 в стене над мусорной урной.

— три вверх поттеры — как мы собираешь убивать себе на себя, не понимаю, что именно следующий метлы, а закрыл рон, с трудом призернил к дурслям. ведь к ним дошего не было, а ты судишь телевезали. в голосе секцию на свете и с трудом выдавил из комнаты о солетание мореди на поле гостиной. а мог выполнить черный метлы, которые они вышел из своих гарри и вернульсь. и не смог поднял сторону и солыбались под нес.

— похожи всем собокой с самой коридором. он был остались подняла собый пол, сотся по своей меня, что он не время забыл о хогвартса?

— да нет… не высобием маглы вернулась по молеть. я вот от небрадитесь, что это очень впереди, что его в воздухе, профессор макгонагалл правел были уже вскативала.

— а ты поймай, мне подумать, — пробормотал он. — квиррелл не мог быть, и представил забыл, ты вот-рон


В общем не так плохо, но в текстах есть много не согласованных друг с другом слов.

## **Model 3** - two GRU

### Обучение

In [ ]:
model.fit(x, y,
          batch_size=128,
          epochs=40)

Epoch 1/40
181265/181265 [==============================] - 159s 876us/step - loss: 0.0094 - acc: 0.2805
Epoch 2/40
181265/181265 [==============================] - 158s 874us/step - loss: 0.0086 - acc: 0.3593
Epoch 3/40
181265/181265 [==============================] - 158s 872us/step - loss: 0.0081 - acc: 0.4094
Epoch 4/40
181265/181265 [==============================] - 157s 867us/step - loss: 0.0076 - acc: 0.4486
Epoch 5/40
181265/181265 [==============================] - 155s 856us/step - loss: 0.0073 - acc: 0.4745
Epoch 6/40
181265/181265 [==============================] - 155s 856us/step - loss: 0.0071 - acc: 0.4950
Epoch 7/40
181265/181265 [==============================] - 159s 875us/step - loss: 0.0069 - acc: 0.5092
Epoch 8/40
181265/181265 [==============================] - 160s 880us/step - loss: 0.0067 - acc: 0.5245
Epoch 9/40
181265/181265 [==============================] - 159s 877us/step - loss: 0.0066 - acc: 0.5363
Epoch 10/40
181265/181265 [============================

В этом эксперименте точность достигла 64%.

### Результаты тренировки

In [ ]:
 print(generateText(800,0.5))

рнон отошел в сторону, а гарри повернулся к ней подаролом в сторону постели на полеский по положение и предпоповой старых сторону, потому что на стену kожет.

— ну #она. — вы подумать, что дадли подумал, что в том произнес гарри, 9то все предпосил гарри.

— 7д, 0»мая поyвали в полном и поднялись по перед.ть, как профессор макгонагалл поняли в 6емных уюсты, в последний водумии камень в qв запрете подорогне столо были за того, что произнес гарри, повернулся к конецу, произнес он. — что не спросилось в коников. — если с ними заметил, что в спальнем cтаг, на возможно, он скозненил произнесил мому 9какой. «bстолько сесть на просто попросил гарри, конечно, 5следив в за ней было 8аконького поправила себя на тор, чтобы подумать, что дадли подумал, что с 8суся, потому что h него не понимает, что он сбыл leобразачивая по*нает просто не бы


В данном примере присутствует много лишних символов, неправильные слова.

Как можно заметить, что сгенерированный текст модели выше (с двумя LSTM) получился более читабельный. Именно на это нам и указывало значение accuracy.

# **Вывод**



Для улучшения результата следует "утяжелить" сети (увеличить число слоёв), делать больше эпох при обучении.